In [80]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [81]:
# Paramètre à modifier : budget total (en M£, ou multiplie x10 si prix en 0.1M£)
my_budget = 1000

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Pondérations pour chaque critère (à ajuster selon l’importance donnée à chaque indicateur)
w_points_per_cost = 0.2
w_ict_index = 0.2
w_minutes_last = 0.1 # positif : favorise joueurs ayant joué récemment
w_total_points_last = 0.3 # positif : favorise joueurs en forme
w_ict_last = 0.3   # positif : favorise joueurs en forme
w_xGI_last = 0.1  # Poids ajouté sur xG_recent pour MID et FWD
w_fdr = -0.2   # négatif : pénalise calendrier difficile
w_total_points_last_per_xGI_last = -0.1 # négatif : met en avant les joueurs sous-côtés

# Variable d’état pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : score multi-indicateurs pondéré
prob = LpProblem("FPL_Optimizer", LpMaximize)
prob += lpSum([
    w_points_per_cost  * df_players.loc[i, 'points_per_cost'] * player_vars[i]
  + w_ict_index       * df_players.loc[i, 'ict_index'] * player_vars[i]
  + w_minutes_last * df_players.loc[i, 'minutes_last'] * player_vars[i]
  + w_total_points_last * df_players.loc[i, 'total_points_last'] * player_vars[i]
  + w_ict_last       * df_players.loc[i, 'ict_last'] * player_vars[i]
  + (w_xGI_last if df_players.iloc[i]['position'] in ['MID', 'FWD'] else 0) * df_players.iloc[i]['xGI_last'] * player_vars[i]
  + w_fdr             * df_players.loc[i, 'fdr_next_6'] * player_vars[i]
  + w_total_points_last_per_xGI_last * df_players.loc[i, 'total_points_last_per_xGI_last'] * player_vars[i]
    for i in range(len(df_players))
    #if player_vars[i] == 1
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte de budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= my_budget

# Contrainte max 3 joueurs de la même équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i] for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Résolution
prob.solve()

# Résultats : joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/3ab287ba4130404ab7339d9c2342e3d7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/3ab287ba4130404ab7339d9c2342e3d7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 31 COLUMNS
At line 5219 RHS
At line 5246 BOUNDS
At line 5988 ENDATA
Problem MODEL has 26 rows, 741 columns and 2964 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 254.91 - 0.00 seconds
Cgl0004I processed model has 26 rows, 612 columns (612 integer (539 of which binary)) and 2448 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 5.77316e-15
Cbc0038I Solution found of -254.91
Cbc0038I Cleaned solution of -254.91
Cbc0038I Before mini branch and bound, 612 integers at bound fixed and 0 continuous

In [82]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,points_per_minutes,fdr_next_6,ict_index,selected_by_percent,selected_rank,form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,assists,goals_scored,goal_involvements,expected_goals,expected_assists,expected_goal_involvements,GI_on_xGI,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90,minutes_last,xGI_last,xG_last,ict_last,total_points_last,total_points_last_per_xGI_last
2,Trevoh,Chalobah,CHE,Chelsea,DEF,52.0,38.0,0.730769,0.088785,2.833333,29.1,10.8,40,8.0,428.0,1202495,256414,238983,44957,1.0,2.0,3.0,0.75,0.55,1.30,2.307692,0.16,0.0,0.12,0.28,1.10,1.05,10.72,85.6,0.260,0.150,5.82,7.6,29.230769
9,Jurriën,J.Timber,ARS,Arsenal,DEF,58.0,35.0,0.603448,0.102639,2.666667,34.4,13.8,28,3.7,341.0,1418551,213767,306345,77497,1.0,2.0,3.0,1.20,0.17,1.37,2.189781,0.32,0.0,0.04,0.36,0.58,0.53,7.92,68.2,0.274,0.240,6.88,7.0,25.547445
34,Daniel,Muñoz,CRY,Crystal Palace,DEF,56.0,26.0,0.464286,0.057778,3.333333,21.8,12.7,31,5.3,450.0,958732,28119,440211,176528,1.0,0.0,1.0,0.36,1.14,1.50,0.666667,0.07,0.0,0.23,0.30,0.94,0.40,5.40,90.0,0.300,0.072,4.36,5.2,17.333333
56,El Hadji Malick,Diouf,WHU,West Ham,DEF,44.0,18.0,0.409091,0.040000,2.833333,26.6,4.0,88,4.7,450.0,81671,15277,401084,82289,3.0,0.0,3.0,0.17,0.65,0.82,3.658537,0.03,0.0,0.13,0.16,1.55,2.60,6.00,90.0,0.164,0.034,5.32,3.6,21.951220
124,Emmanuel,Agbadou,WOL,Wolves,DEF,45.0,12.0,0.266667,0.026667,2.833333,21.9,0.3,300,2.7,450.0,20077,1185,33070,14526,0.0,0.0,0.0,0.38,0.26,0.64,0.000000,0.08,0.0,0.05,0.13,1.54,2.40,10.40,90.0,0.128,0.076,4.38,2.4,18.750000
40,João Pedro,João Pedro,CHE,Chelsea,FWD,78.0,35.0,0.448718,0.083333,2.833333,36.3,67.8,1,6.0,420.0,2582389,290252,973772,215335,3.0,2.0,5.0,1.50,0.20,1.70,2.941176,0.32,0.0,0.04,0.36,1.05,0.86,4.71,84.0,0.340,0.300,7.26,7.0,20.588235
46,Richarlison,Richarlison,TOT,Spurs,FWD,68.0,29.0,0.426471,0.083573,2.666667,38.0,16.7,22,4.0,347.0,2046668,484398,777028,101669,1.0,3.0,4.0,1.64,0.18,1.82,2.197802,0.43,0.0,0.05,0.48,1.34,0.78,5.19,69.4,0.364,0.328,7.60,5.8,15.934066
91,Erling,Haaland,MCI,Man City,FWD,143.0,46.0,0.321678,0.111380,3.000000,52.2,47.0,4,10.3,413.0,2881562,1153854,609982,73077,0.0,6.0,6.0,6.18,0.32,6.50,0.923077,1.35,0.0,0.07,1.42,1.30,0.87,4.36,82.6,1.300,1.236,10.44,9.2,7.076923
3,Robin,Roefs,SUN,Sunderland,GKP,45.0,29.0,0.644444,0.064444,3.000000,13.0,2.6,110,6.7,450.0,293034,67152,84013,17231,0.0,0.0,0.0,0.00,0.00,0.00,NaN,0.00,2.8,0.00,0.00,1.05,0.80,0.00,90.0,0.000,0.000,2.60,5.8,0.000000
48,Martin,Dúbravka,BUR,Burnley,GKP,40.0,17.0,0.425000,0.037778,3.166667,14.3,34.4,5,3.0,450.0,689045,117207,364396,69789,0.0,0.0,0.0,0.00,0.00,0.00,NaN,0.00,3.8,0.00,0.00,2.05,1.60,0.00,90.0,0.000,0.000,2.86,3.4,0.000000


In [83]:
selected_df['price'].sum()

np.float64(1000.0)

In [84]:
selected_df['selected_by_percent'].sum()

np.float64(322.09999999999997)